In [3]:
%matplotlib inline
import numpy as np
import numpy.ma as ma
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
import timeit
import line_profiler

import datetime

from io import StringIO
from pandas.api.types import CategoricalDtype

import pandas._testing as tm

# 2.19 Time series / date functionality

In [10]:
dti = pd.to_datetime(
    ['1.1.2018', np.datetime64('2018-01-01'), datetime.datetime(2018,1,1)]
)
dti[1]

Timestamp('2018-01-01 00:00:00')

In [11]:
dti = pd.date_range('2018-01-01', periods=3, freq='4H')
dti

DatetimeIndex(['2018-01-01 00:00:00', '2018-01-01 04:00:00',
               '2018-01-01 08:00:00'],
              dtype='datetime64[ns]', freq='4H')

In [12]:
dti = dti.tz_localize('Europe/Berlin')


In [13]:
dti.tz_convert('US/Central')

DatetimeIndex(['2017-12-31 17:00:00-06:00', '2017-12-31 21:00:00-06:00',
               '2018-01-01 01:00:00-06:00'],
              dtype='datetime64[ns, US/Central]', freq=None)

In [14]:
idx = pd.date_range("2018-01-01", periods=5, freq="H")
ts = pd.Series(range(len(idx)), index=idx)
print(ts)
ts.resample("2H").count()
print(ts.resample("2H").mean())
ts.resample("2H").median()

2018-01-01 00:00:00    0
2018-01-01 01:00:00    1
2018-01-01 02:00:00    2
2018-01-01 03:00:00    3
2018-01-01 04:00:00    4
Freq: H, dtype: int64
2018-01-01 00:00:00    0.5
2018-01-01 02:00:00    2.5
2018-01-01 04:00:00    4.0
Freq: 2H, dtype: float64


2018-01-01 00:00:00    0.5
2018-01-01 02:00:00    2.5
2018-01-01 04:00:00    4.0
Freq: 2H, dtype: float64

In [15]:
friday = pd.Timestamp("2018-01-05")
friday.day_name()
saturday = friday + pd.Timedelta("1 day")
saturday.day_name() # 'Saturday'

# Add 1 business day (Friday --> Monday)
monday = friday + pd.offsets.BDay()
monday.day_name()

'Monday'

In [16]:
pd.Series(range(3), index=pd.date_range('2020', freq='D', periods=3))


2020-01-01    0
2020-01-02    1
2020-01-03    2
Freq: D, dtype: int64

In [17]:
df = pd.DataFrame({"type A":[15], "type B": [20], "date": ["2012-03-01"], "station": ["s1"]})

In [18]:
df

,type A,type B,date,station
0,15,20,2012-03-01,s1


In [19]:
friday = pd.Timestamp("2018-01-05")
friday.day_name()
friday + pd.Timedelta('1 Day')
(friday + pd.offsets.Day()).day_name()

'Saturday'

In [20]:
s = pd.Series(pd.date_range('2020', periods=3))
df = s.to_frame()

In [21]:
df = pd.Series(pd.date_range('1/13/2021', periods=6)).to_frame()
df.columns=["dates"]
df['day'] = df.dates.dt.day_name()
df['weekday'] = df.dates.dt.weekday
df

,dates,day,weekday
0,2021-01-13,Wednesday,2
1,2021-01-14,Thursday,3
2,2021-01-15,Friday,4
3,2021-01-16,Saturday,5
4,2021-01-17,Sunday,6
5,2021-01-18,Monday,0


In [22]:
pd.Timestamp(pd.NaT)
pd.Timedelta(pd.NaT)
pd.Period(pd.NaT)
pd.DateOffset()

<DateOffset>

In [23]:
pd.Timedelta('14 days')//pd.Timedelta(1, 'W')+1

3

In [24]:
pd.Timestamp("2012-05-01")
pd.Timestamp(datetime.datetime(2012, 5, 1))
pd.Timestamp(2012, 5, 1)

Timestamp('2012-05-01 00:00:00')

In [25]:
pd.Period("2011-01")
pd.Period("2011.01", 'D')
pd.Period("2011.01", freq='D')

Period('2011-01-01', 'D')

In [26]:
dates = pd.date_range('2020/01/01', periods=3)
ts = pd.Series(np.arange(3), dates)
type(ts.index) # pandas.core.indexes.datetimes.DatetimeIndex

pandas.core.indexes.datetimes.DatetimeIndex

In [27]:
periods = pd.date_range('2021-01', periods=3, freq='M')
periods = periods.to_period()
type(periods) # pandas.core.indexes.period.PeriodIndex
ts = pd.Series(np.arange(3), periods)
ts.index # PeriodIndex(['2021-01', '2021-02', '2021-03'], dtype='period[M]', freq='M')
type(ts.index) #pandas.core.indexes.period.PeriodIndex

pandas.core.indexes.period.PeriodIndex

In [28]:
pd.date_range('2018-01', periods=3, freq='M')
pd.period_range('2018-01', periods=3, freq='M')

PeriodIndex(['2018-01', '2018-02', '2018-03'], dtype='period[M]', freq='M')

In [29]:
pd.Period("2011-01", 'D')
pd.Timestamp("2011-01", 'D')

Timestamp('2011-01-01 00:00:00', freq='D')

In [31]:
pd.date_range('2021-01', periods=3, freq='M').to_period()

PeriodIndex(['2021-01', '2021-02', '2021-03'], dtype='period[M]', freq='M')

In [40]:
print(pd.Timestamp('2020.01.01')) # 2020-01-01 00:00:00
pd.to_datetime(['2020.01.01', '2020.01.02'])
pd.date_range('2020.01.01', periods=3, freq='D')

pd.Period('2020.01.01')

2020-01-01 00:00:00


Period('2020-01-01', 'D')

In [50]:
pd.to_datetime(pd.Series(["Jul 31, 2020","2010-01-10", None])) 
pd.to_datetime(["Jul 31, 2020","2010-01-10", None]) # DatetimeIndex(['2020-07-31', '2010-01-10', 'NaT'], dtype='datetime64[ns]', freq=None)

DatetimeIndex(['2020-07-31', '2010-01-10', 'NaT'], dtype='datetime64[ns]', freq=None)

In [55]:
pd.to_datetime('04-01-2021',dayfirst=True)
pd.to_datetime('04-18-2021',dayfirst=True) # Timestamp('2021-04-18 00:00:00')

Timestamp('2021-04-18 00:00:00')

In [61]:
pd.DatetimeIndex(["2018-01-01", "2018-01-03", "2018-01-05"], freq="infer") # DatetimeIndex(['2018-01-01', '2018-01-03', '2018-01-05'], dtype='datetime64[ns]', freq='2D')
pd.DatetimeIndex(["2018-01-01", "2018-01-03", "2018-01-06"], freq="infer") # DatetimeIndex(['2018-01-01', '2018-01-03', '2018-01-06'], dtype='datetime64[ns]', freq=None)

DatetimeIndex(['2018-01-01', '2018-01-03', '2018-01-06'], dtype='datetime64[ns]', freq=None)

In [63]:
pd.to_datetime("2020/11/12", format="%Y/%m/%d")
pd.to_datetime("12-11-2010 00:00", format="%d-%m-%Y %H:%M")

Timestamp('2010-11-12 00:00:00')

In [2]:
pd.Timestamp.max

Timestamp('2262-04-11 23:47:16.854775807')

In [11]:
pd.Timestamp('2021.04.30') + pd.offsets.BQuarterEnd()

Timestamp('2021-06-30 00:00:00')

In [16]:
periods = pd.date_range('2021-01', periods=3, freq='M')
periods.dtype

dtype('<M8[ns]')

In [19]:
pd.Timestamp.max

Timestamp('2262-04-11 23:47:16.854775807')

In [27]:
df = DataFrame({
    'year': [2021, 2021],
    'month': [4,5],
    'day':[30, 1],
    'hours':[23, 1]
})
pd.to_datetime(df[['year', 'month', 'day']]) # ValueError: to assemble mappings requires at least that [year, month, day] be specified: [day] is missing

0   2021-04-30
1   2021-05-01
dtype: datetime64[ns]

In [37]:
pd.to_datetime(['2020/01/01', '2021.01.02', 'asd']) # ParserError: Unknown string format: asd
pd.to_datetime(['2020/01/01', '2021.01.02', 'asd'], errors='raise') # ParserError: Unknown string format: asd
pd.to_datetime(['2020/01/01', '2021.01.02', 'asd'], errors='ignore')
pd.to_datetime(['2020/01/01', '2021.01.02', 'asd'], errors='coerce')

DatetimeIndex(['2020-01-01', '2021-01-02', 'NaT'], dtype='datetime64[ns]', freq=None)

In [46]:
pd.to_datetime([1349720105, 1349806505, 1349892905, 
                1349979305, 1350065705], unit="s")
pd.to_datetime([1349720105100, 1349720105200, 1349720105300, 
                1349720105400,1349720105500], unit='ms')
pd.to_datetime([1618866007.161886600, 1618866007161886601])

DatetimeIndex(['1970-01-01 00:00:01.618866007', '2021-04-19 21:00:07.161886601'], dtype='datetime64[ns]', freq=None)

In [45]:
pd.to_datetime(1490195805433502912, unit="ns")

Timestamp('2017-03-22 15:16:45.433502912')

In [53]:
stamps = pd.date_range("2012-10-08 18:15:05", periods=4, freq="D")
(stamps - pd.Timestamp("1970-01-01")) // pd.Timedelta("1s")

Int64Index([1349720105, 1349806505, 1349892905, 1349979305], dtype='int64')

In [58]:
pd.to_datetime([0, 1, 2], unit="D", origin=pd.Timestamp("2000-01-01"))

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03'], dtype='datetime64[ns]', freq=None)

# 2.19.4 Generating ranges of timestamps

In [4]:
dates = [
    datetime.datetime(2020,1,3),
    datetime.datetime(2020,1,4),
    datetime.datetime(2020,1,5),
]
dates
# Note the frequency information
index = pd.DatetimeIndex(dates)
index # DatetimeIndex(['2020-01-03', '2020-01-04', '2020-01-05'], dtype='datetime64[ns]', freq=None)

# Automatically converted to DatetimeIndex
index = pd.Index(dates)
index # DatetimeIndex(['2020-01-03', '2020-01-04', '2020-01-05'], dtype='datetime64[ns]', freq=None)


DatetimeIndex(['2020-01-03', '2020-01-04', '2020-01-05'], dtype='datetime64[ns]', freq=None)

In [2]:
pd.period_range(start='2020.01.01', end='2020.01.03')

PeriodIndex(['2020-01-01', '2020-01-02', '2020-01-03'], dtype='period[D]', freq='D')

In [10]:
pd.Timestamp.max # Timestamp('2262-04-11 23:47:16.854775807')
pd.Timestamp.min # Timestamp('1677-09-21 00:12:43.145225')

Timestamp('1677-09-21 00:12:43.145225')

In [3]:
start = datetime.datetime(2011, 1, 1)
end = datetime.datetime(2012, 1, 1)

df= pd.date_range(start, end, freq='M')
pd.period_range(start, end, freq='M')
pd.bdate_range(start, end, freq='M')

DatetimeIndex(['2011-01-31', '2011-02-28', '2011-03-31', '2011-04-30',
               '2011-05-31', '2011-06-30', '2011-07-31', '2011-08-31',
               '2011-09-30', '2011-10-31', '2011-11-30', '2011-12-31'],
              dtype='datetime64[ns]', freq='M')

In [13]:
pd.date_range(start, periods=5, freq='QS')
pd.bdate_range(start, periods=5, freq='BQS')
pd.period_range(start, periods=4, freq='W-MON')

PeriodIndex(['2010-12-28/2011-01-03', '2011-01-04/2011-01-10',
             '2011-01-11/2011-01-17', '2011-01-18/2011-01-24'],
            dtype='period[W-MON]', freq='W-MON')

In [17]:
pd.bdate_range(start, end, freq="M")

DatetimeIndex(['2011-01-31', '2011-02-28', '2011-03-31', '2011-04-30',
               '2011-05-31', '2011-06-30', '2011-07-31', '2011-08-31',
               '2011-09-30', '2011-10-31', '2011-11-30', '2011-12-31'],
              dtype='datetime64[ns]', freq='M')

In [4]:
pd.date_range(start, end, freq="BM")

DatetimeIndex(['2011-01-31', '2011-02-28', '2011-03-31', '2011-04-29',
               '2011-05-31', '2011-06-30', '2011-07-29', '2011-08-31',
               '2011-09-30', '2011-10-31', '2011-11-30', '2011-12-30'],
              dtype='datetime64[ns]', freq='BM')

In [22]:
pd.bdate_range(end=end, periods=3, freq='BM')

DatetimeIndex(['2011-10-31', '2011-11-30', '2011-12-30'], dtype='datetime64[ns]', freq='BM')

In [23]:
weekmask = "Mon Wed Fri"
holidays = [datetime.datetime(2011, 1, 5), datetime.datetime(2011, 3, 14)]

In [48]:
pd.bdate_range(datetime.datetime(2011, 1, 1), 
              datetime.datetime(2011, 4, 30), freq='C',
             weekmask=weekmask, holidays=holidays)

DatetimeIndex(['2011-01-03', '2011-01-07', '2011-01-10', '2011-01-12',
               '2011-01-14', '2011-01-17', '2011-01-19', '2011-01-21',
               '2011-01-24', '2011-01-26', '2011-01-28', '2011-01-31',
               '2011-02-02', '2011-02-04', '2011-02-07', '2011-02-09',
               '2011-02-11', '2011-02-14', '2011-02-16', '2011-02-18',
               '2011-02-21', '2011-02-23', '2011-02-25', '2011-02-28',
               '2011-03-02', '2011-03-04', '2011-03-07', '2011-03-09',
               '2011-03-11', '2011-03-16', '2011-03-18', '2011-03-21',
               '2011-03-23', '2011-03-25', '2011-03-28', '2011-03-30',
               '2011-04-01', '2011-04-04', '2011-04-06', '2011-04-08',
               '2011-04-11', '2011-04-13', '2011-04-15', '2011-04-18',
               '2011-04-20', '2011-04-22', '2011-04-25', '2011-04-27',
               '2011-04-29'],
              dtype='datetime64[ns]', freq='C')

In [51]:
pd.Timestamp.min # Timestamp('1677-09-21 00:12:43.145225')
pd.Timestamp.max # Timestamp('2262-04-11 23:47:16.854775807')

Timestamp('2262-04-11 23:47:16.854775807')

In [53]:
df = pd.DataFrame({'Date': ['16234', '16003']})
df['Date'] = pd.to_datetime(df['Date'], format='%y%j')
df

,Date
0,2016-08-21
1,2016-01-03


In [58]:
rng = pd.bdate_range(start, end, freq="BM")
ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts[:5]
ts[:5].index
ts[::2].index

DatetimeIndex(['2011-01-31', '2011-03-31', '2011-05-31', '2011-07-29',
               '2011-09-30', '2011-11-30'],
              dtype='datetime64[ns]', freq='2BM')

In [60]:
ts["1/31/2011"]
ts[datetime.datetime(2011, 12, 25):]

2011-12-30   -0.239682
Freq: BM, dtype: float64

In [64]:
ts['2011']
ts['2011-5']

2011-01-31   -0.100624
2011-02-28   -0.452321
2011-03-31    2.077659
2011-04-29    0.559258
2011-05-31    0.701382
2011-06-30   -0.544770
2011-07-29   -1.489551
2011-08-31   -0.855428
2011-09-30    0.955600
2011-10-31    0.366601
2011-11-30    1.572487
2011-12-30   -0.239682
Freq: BM, dtype: float64

In [74]:
df = ts.to_frame()
df.rename(columns={(0):'col'}, inplace=True)
df['2011-01-31'] # KeyError: '2011-01-31'
df.loc['2011-01-31'] # col   -0.100624

col   -0.100624
Name: 2011-01-31 00:00:00, dtype: float64

In [29]:
dft = pd.DataFrame(
    np.random.randn(100000, 1),
    columns=["A"],
    index=pd.date_range("20130101", periods=100000, freq="T"),
)
dft.loc['2013']
dft["2013-1":"2013-2"]

,A
2013-01-01 00:00:00,0.390870
2013-01-01 00:01:00,0.350306
2013-01-01 00:02:00,1.538080
2013-01-01 00:03:00,-0.394361
2013-01-01 00:04:00,-1.246669
...,...
2013-02-28 23:55:00,0.122760
2013-02-28 23:56:00,1.239373
2013-02-28 23:57:00,-1.572780
2013-02-28 23:58:00,-0.567776


In [92]:
dft2 = pd.DataFrame(
    np.random.randn(20,1),
    columns=['A'],
    index=pd.MultiIndex.from_product([
        pd.date_range("2020.01.01", periods=10, freq='12H'),
        ['a','b']
    ])
)
dft2.loc['20200105']
idx = pd.IndexSlice
dft2 = dft2.swaplevel(0,1).sort_index()
dft2.loc[idx[:,'20200105'], :]
dft2.loc[idx['a','20200105'], :]

A
a 2020-01-05 00:00:00 -0.922086
  2020-01-05 12:00:00  0.619447

In [95]:
df = pd.DataFrame([0], 
        index=pd.DatetimeIndex(["2019-01-01"],tz="US/Pacific"))
df["2019-01-01 12:00:00+04:00":"2019-01-01 13:00:00+04:00"]

,0
2019-01-01 00:00:00-08:00,0


In [96]:
df["2019-01-01 12:00:00+04:00":"2019-01-01 13:00:00+04:00"]

,0
2019-01-01 00:00:00-08:00,0


In [14]:
series_minute = pd.Series(np.arange(1,4), 
            pd.DatetimeIndex(["2011-12-31 23:59:00", "2012-01-01 00:00:00", 
                              "2012-01-01 00:02:00"],))
series_minute.index.resolution # 'minute'
# A timestamp string less accurate than a minute gives a Series object.
series_minute['2011-12-31 23'] # 2011-12-31 23:59:00
series_minute['2012-01-01 00:00'] # 2

2011-12-31 23:59:00    1
dtype: int64

In [24]:
dft_minute = pd.DataFrame(
    {"a": [1, 2, 3], "b": [4, 5, 6]}, index=series_minute.index
)
dft_minute.loc['2012-01-01 00:02']

a    3
b    6
Name: 2012-01-01 00:02:00, dtype: int64

In [28]:
series_monthly = pd.Series(
    [1, 2, 3], pd.DatetimeIndex(["2011-12", "2012-01", "2012-02"])
)
series_monthly.index.resolution # 'day'
series_monthly["2011-12"] # returns Series - 2011-12-01
series_monthly["2011-12-01"] # returns scaler 1

1

In [32]:
dft[datetime.datetime(2013, 1, 1): datetime.datetime(2013, 2, 28)]

,A
2013-01-01 00:00:00,0.390870
2013-01-01 00:01:00,0.350306
2013-01-01 00:02:00,1.538080
2013-01-01 00:03:00,-0.394361
2013-01-01 00:04:00,-1.246669
...,...
2013-02-27 23:56:00,-1.037733
2013-02-27 23:57:00,-1.733930
2013-02-27 23:58:00,0.424122
2013-02-27 23:59:00,1.417177


In [33]:
dft[datetime.datetime(2013, 1, 1, 10, 12, 0): datetime.datetime(2013, 2, 28, 10, 12, 0)]

,A
2013-01-01 10:12:00,-1.417470
2013-01-01 10:13:00,-1.125379
2013-01-01 10:14:00,-1.855770
2013-01-01 10:15:00,-1.651701
2013-01-01 10:16:00,1.253949
...,...
2013-02-28 10:08:00,-0.697029
2013-02-28 10:09:00,2.197959
2013-02-28 10:10:00,-0.673779
2013-02-28 10:11:00,1.285071


In [43]:
rng2 = pd.date_range("2020-01-01", "2021-01-01", freq="W")
rng2
ts2 = pd.Series(np.random.randn(len(rng2)), index=rng2)
ts2.truncate(before='2020-1', after='2020-02') # after month is not included
ts2['2020-01':'2020-02']
ts2[[0,2,6]]
ts2[[0,2,6]].index # Freq is lost
ts2[[0,2,4]].index # Freq is present

DatetimeIndex(['2020-01-05', '2020-01-19', '2020-02-02'], dtype='datetime64[ns]', freq='2W-SUN')

In [48]:
idx = pd.date_range(start="2019-12-29", freq="D", periods=4)
idx.isocalendar()
idx.to_series().dt.isocalendar()

,year,week,day
2019-12-29,2019,52,7
2019-12-30,2020,1,1
2019-12-31,2020,1,2
2020-01-01,2020,1,3


In [54]:
friday = pd.Timestamp('2021.05.07')
friday.day_name()

# Add 2 business days (Friday --> Tuesday)
two_business_days = 2 * pd.offsets.BDay()
two_business_days.apply(friday)
two_business_days + friday
(two_business_days + friday).day_name()

'Tuesday'

In [59]:
ts = pd.Timestamp("2021-05-08 00:00:00")

# BusinessHour's valid offset dates are Monday through Friday
offset = pd.offsets.BusinessHour(start="09:00")

# Bring the date to the closest offset date (Monday
offset.rollforward(ts) # Timestamp('2018-01-08 09:00:00')

# Date is brought to the closest offset date first and then the hour is added
ts + offset # Timestamp('2021-05-10 10:00:00')

Timestamp('2021-05-10 10:00:00')

In [67]:
ts = pd.Timestamp("2014-01-01 09:00")
day = pd.offsets.Day()
day.apply(ts) # Timestamp('2014-01-02 09:00:00')
day.apply(ts).normalize() # Timestamp('2014-01-02 00:00:00')

ts = pd.Timestamp("2014-01-01 22:00")
hour = pd.offsets.Hour()
(ts + hour).normalize() # Timestamp('2014-01-01 00:00:00')

hour.apply(pd.Timestamp("2014-01-01 23:30")).normalize() # Timestamp('2014-01-02 00:00:00')

Timestamp('2014-01-02 00:00:00')

In [70]:
dfpd.Timestamp('2014-01-02 01:00:00').normalize()

Timestamp('2014-01-02 00:00:00')

In [6]:
pd.offsets.Day(1)

<Day>

In [26]:
d = datetime.datetime(2021, 5, 4, 9, 0)
d = pd.Timestamp('2021.05.04 09:00')
d + pd.offsets.Week(weekday=4) # Timestamp('2021-05-07 09:00:00')
(d + pd.offsets.Week(weekday=4)).weekday() # 4
(d + pd.offsets.Week(weekday=4)).day_name() # 'Friday'
(d + pd.offsets.Week(weekday=4)).month_name() # 'May'


AttributeError: 'Timestamp' object has no attribute 'month_day'

In [29]:
d - pd.offsets.Week(normalize=True) # Timestamp('2021-05-11 00:00:00')
d - pd.offsets.Week(normalize=True) # Timestamp('2021-04-27 00:00:00')

Timestamp('2021-04-27 00:00:00')

In [34]:
d + pd.offsets.YearEnd() # Timestamp('2021-12-31 09:00:00')
d + pd.offsets.YearEnd(normalize=True) # Timestamp('2021-12-31 00:00:00')
d + pd.offsets.YearEnd(month=6) # Timestamp('2021-06-30 09:00:00')

Timestamp('2021-06-30 09:00:00')

In [72]:
rng = pd.date_range("2012-01-01", "2012-01-03")
s = pd.Series(rng)
rng + pd.DateOffset(months=2)
s + pd.DateOffset(months=2)
s - pd.DateOffset(months=2)

s - pd.offsets.Day(2)
td = s - pd.Series(pd.date_range('2011.12.28', '2011.12.30'))
td + pd.offsets.Minute(15)

0   4 days 00:15:00
1   4 days 00:15:00
2   4 days 00:15:00
dtype: timedelta64[ns]

In [58]:
d = pd.date_range('2021.05.04', periods=2)
d + pd.DateOffset(months=1) ## DatetimeIndex(['2021-06-04', '2021-06-05'], dtype='datetime64[ns]', freq=None)
d + pd.offsets.MonthEnd() ## DatetimeIndex(['2021-05-31', '2021-05-31'], dtype='datetime64[ns]', freq=None)

DatetimeIndex(['2021-05-31', '2021-05-31'], dtype='datetime64[ns]', freq=None)

In [63]:
d - pd.offsets.Day(2) ## DatetimeIndex(['2021-05-02', '2021-05-03'], dtype='datetime64[ns]', freq=None)
d - pd.DateOffset(2) ## DatetimeIndex(['2021-05-02', '2021-05-03'], dtype='datetime64[ns]', freq=None)

DatetimeIndex(['2021-05-02', '2021-05-03'], dtype='datetime64[ns]', freq=None)

In [73]:
rng + pd.offsets.BQuarterEnd()

DatetimeIndex(['2012-03-30', '2012-03-30', '2012-03-30'], dtype='datetime64[ns]', freq=None)

In [81]:
bh = pd.offsets.BusinessHour()
# 2014-08-01 is Friday
pd.Timestamp("2014-08-01 10:00").weekday() # 4
pd.Timestamp("2014-08-01 10:00") + bh # Timestamp('2014-08-01 11:00:00')

# Below example is the same as: pd.Timestamp('2014-08-01 09:00') + bh
pd.Timestamp("2014-08-01 08:00") + bh # Timestamp('2014-08-01 10:00:00')

# If the results is on the end time, move to the next business day
pd.Timestamp("2014-08-01 16:00") + bh # Timestamp('2014-08-04 09:00:00')

## Remainings are added to the next day
pd.Timestamp("2014-08-01 16:30") + bh # Timestamp('2014-08-04 09:30:00')

# Adding 2 business hours
pd.Timestamp("2014-08-01 10:00") + pd.offsets.BusinessHour(2)

# Subtracting 3 business hours
pd.Timestamp("2021-05-03 10:00") + pd.offsets.BusinessHour(-3) # Timestamp('2021-04-30 15:00:00')


Timestamp('2021-04-30 15:00:00')

In [9]:
pd.date_range(start, periods=6, freq="B")
pd.date_range(start, periods=6, freq=pd.offsets.BDay())

DatetimeIndex(['2011-01-03', '2011-01-04', '2011-01-05', '2011-01-06',
               '2011-01-07', '2011-01-10'],
              dtype='datetime64[ns]', freq='B')

In [12]:
pd.date_range(start, periods=15, freq="2h20min")
pd.date_range(start, periods=10, freq="1D10U")

DatetimeIndex([       '2011-01-01 00:00:00', '2011-01-02 00:00:00.000010',
               '2011-01-03 00:00:00.000020', '2011-01-04 00:00:00.000030',
               '2011-01-05 00:00:00.000040', '2011-01-06 00:00:00.000050',
               '2011-01-07 00:00:00.000060', '2011-01-08 00:00:00.000070',
               '2011-01-09 00:00:00.000080', '2011-01-10 00:00:00.000090'],
              dtype='datetime64[ns]', freq='86400000010U')

In [25]:
pd.Timestamp("2014-1-2") + pd.offsets.MonthBegin(n=1) # Timestamp('2014-02-01 00:00:00')
pd.Timestamp("2014-1-2") + pd.offsets.MonthEnd(n=1) # Timestamp('2014-01-31 00:00:00')
pd.Timestamp("2014-1-2") - pd.offsets.MonthBegin(n=1) # Timestamp('2014-01-01 00:00:00')
pd.Timestamp("2014-1-2") - pd.offsets.MonthEnd(n=1) # Timestamp('2013-12-31 00:00:00')
pd.Timestamp("2014-1-2") + pd.offsets.MonthBegin(n=4) # Timestamp('2014-05-01 00:00:00')
pd.Timestamp("2014-1-2") - pd.offsets.MonthEnd(n=4) # Timestamp('2013-09-30 00:00:00')

Timestamp('2013-09-30 00:00:00')

In [129]:
rng = pd.date_range("2012-01-01", "2012-01-03")
ts = pd.Series(range(len(rng)), index=rng)
ts[:5]
ts.shift(1)
ts.shift(1).bfill()
ts.shift(5, freq='D')
ts.shift(5, freq=pd.offsets.BDay())
ts.shift(5, freq="BM")
ts.shift(1, fill_value=10)

2012-01-01    10
2012-01-02     0
2012-01-03     1
Freq: D, dtype: int64

In [50]:
(ts.shift(-1)+ts).ffill()
(ts.shift(1, freq='D') + ts).ffill().bfill()

2012-01-01    1.0
2012-01-02    1.0
2012-01-03    3.0
2012-01-04    3.0
Freq: D, dtype: float64

In [78]:
df = pd.DataFrame({
    'date':pd.to_datetime('2021.05.01, 2021.05.01, 2021.05.3, 2021.05.4'.split(',')),
    'val':[10,20,12,15]
})

df.groupby('date').val.agg(['min', 'max', 'mean']).asfreq('3D')

,min,max,mean
date,,,
2021-05-01,10,20,15
2021-05-04,15,15,15


In [103]:
d="""timestamp,v
1,5
2,6
6,7
8,8"""
df=pd.read_csv(StringIO(d))
df

,timestamp,v
0,1,5
1,2,6
2,6,7
3,8,8


In [104]:
#df = df.set_index('timestamp')
#df.reindex(np.arange(df.index[0], 1+df.index[-1])).ffill().reset_index().astype(int)

In [123]:
dr = pd.date_range(start='2010.1.1', periods=3, freq=3*pd.offsets.BDay())
ts = pd.Series(np.random.randn(3), index=dr)
ts.asfreq(pd.offsets.BDay())
ts.asfreq(pd.DateOffset(1))
ts.asfreq(pd.offsets.BDay()).ffill()
ts.asfreq(pd.DateOffset(), method='pad')
rng.to_pydatetime()

array([datetime.datetime(2012, 1, 1, 0, 0),
       datetime.datetime(2012, 1, 2, 0, 0),
       datetime.datetime(2012, 1, 3, 0, 0)], dtype=object)

# Resampling

In [34]:
rng = pd.date_range("1/1/2012", periods=400, freq="S")
ts = pd.Series(np.random.randint(0, 500, len(rng)), index=rng)
ts.resample('5min').sum()
ts.resample('5 Min').mean()
ts.resample('5 min').ohlc() ## Open high low close
ts.resample('5 min').max()

ts.resample('5 min', closed='right').max()

2011-12-31 23:55:00    139
2012-01-01 00:00:00    499
2012-01-01 00:05:00    499
Freq: 5T, dtype: int64

In [26]:
ts.resample('5 min').sum()

2012-01-01 00:00:00    21
2012-01-01 00:05:00    10
Freq: 5T, dtype: int64

In [28]:
ts.resample('5 min', closed='right').sum()

2011-12-31 23:55:00     8
2012-01-01 00:00:00    18
2012-01-01 00:05:00     5
Freq: 5T, dtype: int64

In [31]:
rng = pd.date_range("1/1/2012", periods=7, freq="T")
ts = pd.Series(np.random.randint(0, 10, len(rng)), index=rng)
print(ts)

2012-01-01 00:00:00    4
2012-01-01 00:01:00    3
2012-01-01 00:02:00    3
2012-01-01 00:03:00    0
2012-01-01 00:04:00    4
2012-01-01 00:05:00    6
2012-01-01 00:06:00    3
Freq: T, dtype: int64


In [32]:
ts.resample('5 min').sum()

2012-01-01 00:00:00    14
2012-01-01 00:05:00     9
Freq: 5T, dtype: int64

In [33]:
ts.resample('5 min', label='right').sum()

2012-01-01 00:05:00    14
2012-01-01 00:10:00     9
Freq: 5T, dtype: int64

In [22]:
s = pd.date_range("2021-05-08", "2021-05-12").to_series()
s.iloc[2] = pd.NaT
print(s)
s.dt.day_name()
# default: label='left', closed='left'
s.resample('B', label='right', closed='right', kind='period', convention='end').last().dt.day_name()

2021-05-08   2021-05-08
2021-05-09   2021-05-09
2021-05-10          NaT
2021-05-11   2021-05-11
2021-05-12   2021-05-12
Freq: D, dtype: datetime64[ns]


2021-05-10       Sunday
2021-05-11      Tuesday
2021-05-12    Wednesday
Freq: B, dtype: object

In [31]:
rng = pd.date_range("1/1/2012", periods=7, freq="T")
ts = pd.Series(np.random.randint(0, 10, len(rng)), index=rng)
print(ts)
ts[:2].resample('250ms').asfreq()
ts[:2].resample('250L').ffill()
ts[:2].resample('250L').ffill(limit=2)

2012-01-01 00:00:00    5
2012-01-01 00:01:00    3
2012-01-01 00:02:00    7
2012-01-01 00:03:00    8
2012-01-01 00:04:00    9
2012-01-01 00:05:00    2
2012-01-01 00:06:00    9
Freq: T, dtype: int64


2012-01-01 00:00:00.000    5.0
2012-01-01 00:00:00.250    5.0
2012-01-01 00:00:00.500    5.0
2012-01-01 00:00:00.750    NaN
2012-01-01 00:00:01.000    NaN
                          ... 
2012-01-01 00:00:59.000    NaN
2012-01-01 00:00:59.250    NaN
2012-01-01 00:00:59.500    NaN
2012-01-01 00:00:59.750    NaN
2012-01-01 00:01:00.000    3.0
Freq: 250L, Length: 241, dtype: float64

In [38]:
rng = pd.date_range('2021.01.01', periods=100, freq='D')+pd.Timedelta('1s')
ts = pd.Series(range(100), index=rng)
print(ts.head())
ts.resample('3T').sum()

2021-01-01 00:00:01    0
2021-01-02 00:00:01    1
2021-01-03 00:00:01    2
2021-01-04 00:00:01    3
2021-01-05 00:00:01    4
Freq: D, dtype: int64


2021-01-01 00:00:00     0
2021-01-01 00:03:00     0
2021-01-01 00:06:00     0
2021-01-01 00:09:00     0
2021-01-01 00:12:00     0
                       ..
2021-04-09 23:48:00     0
2021-04-09 23:51:00     0
2021-04-09 23:54:00     0
2021-04-09 23:57:00     0
2021-04-10 00:00:00    99
Freq: 3T, Length: 47521, dtype: int64

In [39]:
from functools import partial
from pandas.tseries.frequencies import to_offset

def round(t, freq):
    freq=to_offset(freq)
    return pd.Timestamp((t.value // freq.delta.value) * freq.delta.value)

ts.groupby(partial(round, freq='3T')).sum()

2021-01-01     0
2021-01-02     1
2021-01-03     2
2021-01-04     3
2021-01-05     4
              ..
2021-04-06    95
2021-04-07    96
2021-04-08    97
2021-04-09    98
2021-04-10    99
Length: 100, dtype: int64

In [51]:
df = pd.DataFrame(
    np.random.randn(1000,3),
    index=pd.date_range('2021.01.01', periods=1000, freq='S'),
    columns=['A','B','C']
)
r = df.resample('3T')
r.mean()

r.A.mean()
r[['A','B']].mean()

r.A.agg([np.sum, np.mean, np.std])
r.agg([np.sum, np.mean])

r.agg({'A':np.sum, 'B':lambda x:np.std(x, ddof=1)})

r.agg({'A':['sum', 'std'], 'B':['mean', 'std']})

A                   B          
                           sum       std      mean       std
2021-01-01 00:00:00  14.400662  1.001800  0.022925  1.019458
2021-01-01 00:03:00   7.374114  1.040298  0.097669  0.953981
2021-01-01 00:06:00  -2.553018  1.033722  0.017430  0.946406
2021-01-01 00:09:00   3.588850  0.994004  0.018482  0.966622
2021-01-01 00:12:00 -10.026925  0.936898 -0.013714  1.104745
2021-01-01 00:15:00   5.662840  0.975147 -0.168076  0.988928

In [54]:
df = pd.DataFrame({
    "date": pd.date_range("2015-01-01", freq="W", periods=5), 
    "a": np.arange(5)},
    index=pd.MultiIndex.from_arrays([[1, 2, 3, 4, 5], pd.date_range("2015-01-01", freq="W", periods=5)],
        names=["v", "d"],)
)
df.resample('M',on='date').sum()


,a
date,
2015-01-31,6
2015-02-28,4


In [5]:
small = pd.Series(range(6),
                 index=pd.to_datetime([
                     "2017-01-01T00:00:00",
                     "2017-01-01T00:30:00",
                     "2017-01-01T00:31:00",
                     "2017-01-01T01:00:00",
                     "2017-01-01T03:00:00",
                     "2017-01-01T03:05:00",
                 ]))
resampled = small.resample('H')
for name, group in resampled:
    print("Group: ", name)
    print("-"*27)
    print(group, end="\n\n")

Group:  2017-01-01 00:00:00
---------------------------
2017-01-01 00:00:00    0
2017-01-01 00:30:00    1
2017-01-01 00:31:00    2
dtype: int64

Group:  2017-01-01 01:00:00
---------------------------
2017-01-01 01:00:00    3
dtype: int64

Group:  2017-01-01 02:00:00
---------------------------
Series([], dtype: int64)

Group:  2017-01-01 03:00:00
---------------------------
2017-01-01 03:00:00    4
2017-01-01 03:05:00    5
dtype: int64



In [7]:
start, end = "2000-10-01 23:30:00", "2000-10-02 00:30:00"
middle = "2000-10-02 00:00:00"
rng = pd.date_range(start, end, freq="7min")
ts = pd.Series(np.arange(len(rng)) * 3, index=rng)
ts

2000-10-01 23:30:00     0
2000-10-01 23:37:00     3
2000-10-01 23:44:00     6
2000-10-01 23:51:00     9
2000-10-01 23:58:00    12
2000-10-02 00:05:00    15
2000-10-02 00:12:00    18
2000-10-02 00:19:00    21
2000-10-02 00:26:00    24
Freq: 7T, dtype: int64

In [9]:
ts.resample('17min', origin="start_day").sum()

2000-10-01 23:14:00     0
2000-10-01 23:31:00     9
2000-10-01 23:48:00    21
2000-10-02 00:05:00    54
2000-10-02 00:22:00    24
Freq: 17T, dtype: int64

In [12]:
ts[middle:end].resample('17min', origin="start_day").sum()

2000-10-02 00:00:00    33
2000-10-02 00:17:00    45
Freq: 17T, dtype: int64

In [14]:
ts.resample("17min", origin="epoch").sum()
ts[middle:end].resample("17min", origin="epoch").sum()

2000-10-01 23:52:00    15
2000-10-02 00:09:00    39
2000-10-02 00:26:00    24
Freq: 17T, dtype: int64

# 2.19.11 Time span representation

In [50]:
p = pd.Period("2014-07-01 09:00", freq="H")
p + pd.offsets.Hour(1)
p + pd.Timedelta(minutes=120)

Period('2014-07-01 11:00', 'H')

In [53]:
pi = pd.period_range("2016-01", periods=3, freq="M")
pi.dtype # period[M]
pi.astype("period[D]")

PeriodIndex(['2016-01-31', '2016-02-29', '2016-03-31'], dtype='period[D]', freq='D')

In [24]:
idx = pd.period_range('2010.12.01', periods=90, freq='D')
ps = pd.Series(np.random.randn(len(idx)), idx)
ps["2011-01"]
ps.index = ps.index.astype('Period[M]')
ps["2011-01"]

2011-01   -1.784246
2011-01    1.002748
2011-01   -0.245326
2011-01   -0.460938
2011-01   -0.768607
2011-01    0.358795
2011-01    0.285641
2011-01    1.091806
2011-01   -0.228718
2011-01    0.086235
2011-01    1.443418
2011-01    2.123803
2011-01   -1.243685
2011-01    1.084482
2011-01   -0.632281
2011-01    0.350900
2011-01    1.755111
2011-01   -1.435417
2011-01   -0.270419
2011-01    1.114668
2011-01   -1.195593
2011-01    1.053937
2011-01   -1.521411
2011-01    0.375012
2011-01   -0.022119
2011-01    0.406801
2011-01    1.454101
2011-01    0.030884
2011-01    1.377780
2011-01    0.524127
2011-01   -0.405650
Freq: M, dtype: float64

In [25]:
prng = pd.period_range("1/1/2011", "1/1/2012", freq="M")
ps = pd.Series(np.random.randn(len(prng)), prng)
ps
ps[datetime.datetime(2011, 12, 25):]

2011-12    0.758171
2012-01    1.057903
Freq: M, dtype: float64

In [28]:
dfp = pd.DataFrame(
    np.random.randn(600, 1),
    columns=["A"],
    index=pd.period_range("2013-01-01 9:00", periods=600, freq="T"),
)
dfp
dfp.loc["2013-01-01 10H"]
dfp["2013-01-01 10H":"2013-01-01 11H"]


,A
2013-01-01 10:00,-0.669510
2013-01-01 10:01,1.503684
2013-01-01 10:02,2.527649
2013-01-01 10:03,0.614396
2013-01-01 10:04,0.267301
...,...
2013-01-01 11:55,-0.107227
2013-01-01 11:56,0.238709
2013-01-01 11:57,-0.334831
2013-01-01 11:58,-0.690552


In [29]:
p = pd.Period("2011", freq="A-DEC")
p

Period('2011', 'A-DEC')

In [48]:
rng = pd.date_range("1/1/2012", periods=5, freq="M")
ts = pd.Series(np.random.randn(len(rng)), index=rng)
ps = ts.to_period()
ps.to_timestamp('D',how='e')
ps.to_timestamp('D',how='e').index.date
ps.to_timestamp("D", how="s")

2012-01-01    0.669606
2012-02-01   -1.298809
2012-03-01    0.125460
2012-04-01   -0.089577
2012-05-01   -1.784285
Freq: MS, dtype: float64

In [50]:
df = pd.DataFrame({'power':np.arange(1.0,9.0)},index=pd.date_range(start='2020-05-04 08:00:00', freq='1h', periods=8))
df

,power
2020-05-04 08:00:00,1.0
2020-05-04 09:00:00,2.0
2020-05-04 10:00:00,3.0
2020-05-04 11:00:00,4.0
2020-05-04 12:00:00,5.0
2020-05-04 13:00:00,6.0
2020-05-04 14:00:00,7.0
2020-05-04 15:00:00,8.0


In [51]:
df['power'].resample('1D').sum()  

2020-05-04    36.0
Freq: D, Name: power, dtype: float64

In [52]:
span=pd.period_range("1215.01.01", '1381.01.01', freq='D')
span

PeriodIndex(['1215-01-01', '1215-01-02', '1215-01-03', '1215-01-04',
             '1215-01-05', '1215-01-06', '1215-01-07', '1215-01-08',
             '1215-01-09', '1215-01-10',
             ...
             '1380-12-23', '1380-12-24', '1380-12-25', '1380-12-26',
             '1380-12-27', '1380-12-28', '1380-12-29', '1380-12-30',
             '1380-12-31', '1381-01-01'],
            dtype='period[D]', length=60632, freq='D')

In [60]:
s=pd.Series([20121231, 20141130, 99991231])

def conv(x):
    return pd.Period(year=x//10000, month=x/100%100, day=x%100, freq='D')

s.apply(conv)

0    2012-12-31
1    2014-11-30
2    9999-12-31
dtype: period[D]

In [59]:
20141130%100

30

In [61]:
span = pd.PeriodIndex(s.apply(conv))
span

PeriodIndex(['2012-12-31', '2014-11-30', '9999-12-31'], dtype='period[D]', freq='D')